## 会話

In [ ]:
http://botpress-docs.s3-website-us-east-1.amazonaws.com/docs/11.9.6/channels/converse/

https://docs.anthropic.com/en/docs/tool-use-examples

https://qiita.com/moritalous/items/8b1a15a7dc583fa3a2e1?utm_campaign=popular_items&utm_medium=twitter&utm_source=dlvr.it

https://community.aws/content/2dtauBCeDa703x7fDS9Q30MJoBA/amazon-bedrock-converse-api-developer-guide

https://qiita.com/moritalous/items/cde191320abcfffacaca?utm_campaign=post_article&utm_medium=twitter&utm_source=twitter_share

https://qiita.com/moritalous/items/8b1a15a7dc583fa3a2e1?utm_campaign=post_article&utm_medium=twitter&utm_source=twitter_share

https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse_stream.html

In [14]:
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "top_song",
                "description": "Get the most popular song played on a radio station.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "sign": {
                                "type": "string",
                                "description": "The call sign for the radio station for which you want the most popular song. Example calls signs are WZPZ, and WKRP."
                            }
                        },
                        "required": [
                            "sign"
                        ]
                    }
                }
            }
        }
    ],
    # "toolChoice": {"any": {"name": "top_song"}}
}

In [23]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0



import boto3
from pprint import pprint

model_id = "anthropic.claude-3-haiku-20240307-v1:0"
system_prompt = "あなたは多くのデータにアクセス可能な経済学者です。日本語で回答せよ。"
prompt = "WZPZで最も人気のある曲は何ですか？"

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name='us-west-2')


# Message to send.
message = {
    "role": "user",
    "content": [{"text": prompt}]
}
messages = [message]
system_prompts = [{"text" : system_prompt}]

# Inference parameters to use.
temperature = 0.5
top_k = 200

# Base inference parameters to use.
inference_config = {"temperature": temperature}
# Additional inference parameters to use.
additional_model_fields = {"top_k": top_k}

# Send the message.
response = bedrock_client.converse(
    modelId=model_id,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config,
    additionalModelRequestFields=additional_model_fields,
    toolConfig=tool_config
)

pprint(response)
print("=" * 30)


output_message = response['output']['message']

print(f"Role: {output_message['role']}")

from pprint import pprint
pprint(output_message)
# for content in output_message['content']:
#     print(f"Text: {content['text']}")

# token_usage = response['usage']
# print(f"Input tokens:  {token_usage['inputTokens']}")
# print(f"Output tokens:  {token_usage['outputTokens']}")
# print(f"Total tokens:  {token_usage['totalTokens']}")
# print(f"Stop reason: {response['stopReason']}")


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '272',
                                      'content-type': 'application/json',
                                      'date': 'Sat, 01 Jun 2024 00:44:22 GMT',
                                      'x-amzn-requestid': '899f0521-3124-4c58-be05-00729314249d'},
                      'HTTPStatusCode': 200,
                      'RequestId': '899f0521-3124-4c58-be05-00729314249d',
                      'RetryAttempts': 0},
 'metrics': {'latencyMs': 724},
 'output': {'message': {'content': [{'toolUse': {'input': {'sign': 'WZPZ'},
                                                 'name': 'top_song',
                                                 'toolUseId': 'tooluse_i9ZmTKM-SgKWt4107tVxRw'}}],
                        'role': 'assistant'}},
 'stopReason': 'tool_use',
 'usage': {'inputTokens': 418, 'outputTokens': 56, 'totalTokens': 474}}
Role: assistant
{'content': [{'too

```json
{
  "content": [
    {
      "toolUse": {
        "input": { "sign": "WZPZ" },
        "name": "top_song",
        "toolUseId": "tooluse_JicO58plTsab0GUsauqcMg"
      }
    }
  ],
  "role": "assistant"
}
```


In [22]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
"""
Shows how to use tools with the Conversation API and the Cohere Command R model.
"""

import logging
import boto3


from botocore.exceptions import ClientError


class StationNotFoundError(Exception):
    """Raised when a radio station isn't found."""
    pass


logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


def get_top_song(call_sign):
    """Returns the most popular song for the requested station.
    Args:
        call_sign (str): The call sign for the station for which you want
        the most popular song.

    Returns:
        response (json): The most popular song and artist.
    """

    song = ""
    artist = ""
    if call_sign == 'WZPZ':
        song = "Elemental Hotel"
        artist = "8 Storey Hike"

    else:
        raise StationNotFoundError(f"Station {call_sign} not found.")

    return song, artist


def generate_text(bedrock_client, model_id, tool_config, input_text):
    """Generates text using the supplied Amazon Bedrock model. If necessary,
    the function handles tool use requests and sends the result to the model.
    Args:
        bedrock_client: The Boto3 Bedrock runtime client.
        model_id (str): The Amazon Bedrock model ID.
        tool_config (dict): The tool configuration.
        input_text (str): The input text.
    Returns:
        Nothing.
    """

    logger.info("Generating text with model %s", model_id)

   # Create the initial message from the user input.
    messages = [{
        "role": "user",
        "content": [{"text": input_text}]
    }]

    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages,
        toolConfig=tool_config
    )

    output_message = response['output']['message']
    messages.append(output_message)
    stop_reason = response['stopReason']

    if stop_reason == 'tool_use':
        # Tool use requested. Call the tool and send the result to the model.
        tool_requests = response['output']['message']['content']
        for tool_request in tool_requests:
            if 'toolUse' in tool_request:
                tool = tool_request['toolUse']
                logger.info("Requesting tool %s. Request: %s",
                            tool['name'], tool['toolUseId'])

                if tool['name'] == 'top_song':
                    tool_result = {}
                    try:
                        song, artist = get_top_song(tool['input']['sign'])
                        tool_result = {
                            "toolUseId": tool['toolUseId'],
                            "content": [{"json": {"song": song, "artist": artist}}]
                        }
                    except StationNotFoundError as err:
                        tool_result = {
                            "toolUseId": tool['toolUseId'],
                            "content": [{"text":  err.args[0]}],
                            "status": 'error'
                        }

                    tool_result_message = {
                        "role": "user",
                        "content": [
                            {
                                "toolResult": tool_result

                            }
                        ]
                    }
                    messages.append(tool_result_message)

                    # Send the tool result to the model.
                    response = bedrock_client.converse(
                        modelId=model_id,
                        messages=messages,
                        toolConfig=tool_config
                    )
                    output_message = response['output']['message']

    # print the final response from the model.
    for content in output_message['content']:
        print(f"{content['text']}")


def main():
    """
    Entrypoint for tool use example.
    """

    logging.basicConfig(level=logging.INFO,
                        format="%(levelname)s: %(message)s")

    model_id = "anthropic.claude-3-haiku-20240307-v1:0"
    input_text = "What is the most popular song on WZPZ?"

    tool_config = {
        "tools": [
            {
                "toolSpec": {
                    "name": "top_song",
                    "description": "Get the most popular song played on a radio station.",
                    "inputSchema": {
                        "json": {
                            "type": "object",
                            "properties": {
                                "sign": {
                                    "type": "string",
                                    "description": "The call sign for the radio station for which you want the most popular song. Example calls signs are WZPZ, and WKRP."
                                }
                            },
                            "required": [
                                "sign"
                            ]
                        }
                    }
                }
            }
        ]
    }
    bedrock_client = boto3.client(service_name='bedrock-runtime', region_name='us-west-2')

    try:
        print(f"Question: {input_text}")
        generate_text(bedrock_client, model_id, tool_config, input_text)

    except ClientError as err:
        message = err.response['Error']['Message']
        logger.error("A client error occurred: %s", message)
        print(f"A client error occured: {message}")

    else:
        print(
            f"Finished generating text with model {model_id}.")


if __name__ == "__main__":
    main()


INFO:__main__:Generating text with model anthropic.claude-3-haiku-20240307-v1:0


Question: What is the most popular song on WZPZ?


INFO:__main__:Requesting tool top_song. Request: tooluse_d8ihx_O_QJi8j6qMfWRGjQ


According to the top_song function, the most popular song played on radio station WZPZ is "Elemental Hotel" by the artist "8 Storey Hike".
Finished generating text with model anthropic.claude-3-haiku-20240307-v1:0.


In [17]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
"""
Shows how to use the Conversation API with Anthropic Claude 3 Sonnet (on demand).
"""

import logging
import boto3


from botocore.exceptions import ClientError


logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


def generate_conversation(bedrock_client,
                     model_id,
                     system_text,
                     input_text):
    """
    Sends a message to a model.
    Args:
        bedrock_client: The Boto3 Bedrock runtime client.
        model_id (str): The model ID to use.
        system_text (JSON) : The system prompt.
        input text : The input message.

    Returns:
        response (JSON): The conversation that the model generated.

    """

    logger.info("Generating message with model %s", model_id)

    # Message to send.
    message = {
        "role": "user",
        "content": [{"text": input_text}]
    }
    messages = [message]
    system_prompts = [{"text" : system_text}]

    # Inference parameters to use.
    temperature = 0.5
    top_k = 200

    #Base inference parameters to use.
    inference_config = {"temperature": temperature}
    # Additional inference parameters to use.
    additional_model_fields = {"top_k": top_k}

    # Send the message.
    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages,
        system=system_prompts,
        inferenceConfig=inference_config,
        additionalModelRequestFields=additional_model_fields
    )

    return response



"""
Entrypoint for Anthropic Claude 3 Sonnet example.
"""

logging.basicConfig(level=logging.INFO,
                    format="%(levelname)s: %(message)s")

model_id = "anthropic.claude-3-haiku-20240307-v1:0"
system_prompt = "あなたは大量のデータにアクセスできる日本人のエコノミストです。"
prompt = "高インフレが国のGDPに与える影響について1000文字で書いてください。"

try:

    bedrock_client = boto3.client(service_name='bedrock-runtime', region_name='us-west-2')

    response = generate_conversation(
        bedrock_client, model_id, system_prompt, prompt)

    output_message = response['output']['message']

    print(f"Role: {output_message['role']}")

    for content in output_message['content']:
        print(f"Text: {content['text']}")

    token_usage = response['usage']
    print(f"Input tokens:  {token_usage['inputTokens']}")
    print(f"Output tokens:  {token_usage['outputTokens']}")
    print(f"Total tokens:  {token_usage['totalTokens']}")
    print(f"Stop reason: {response['stopReason']}")

except ClientError as err:
    message = err.response['Error']['Message']
    logger.error("A client error occurred: %s", message)
    print(f"A client error occured: {message}")

else:
    print(
        f"Finished generating text with model {model_id}.")


INFO:__main__:Generating message with model anthropic.claude-3-haiku-20240307-v1:0


Role: assistant
Text: 高インフレが国のGDPに与える影響について、1000文字でお書きします。

高インフレは、国のGDPに深刻な影響を及ぼします。まず、高インフレは消費者の購買力を低下させます。物価が上昇すれば、同じ金額で以前より少ない商品しか購入できなくなります。これにより、個人消費が減少し、GDP成長率の鈍化につながります。

また、高インフレは企業の生産活動にも悪影響を及ぼします。原材料や人件費の高騰により、企業の利益率が低下します。利益が減少すれば、企業は設備投資の抑制や雇用調整を余儀なくされ、経済全体の生産性が低下します。

さらに、高インフレは国際競争力の低下にもつながります。自国通貨の価値が下落すれば、輸出製品の価格が相対的に高くなり、海外市場での競争力が失われます。一方で、輸入品の価格が上昇するため、国内需要も減少します。これらの要因により、貿易収支の悪化を招き、GDPの減少につながります。

加えて、高インフレは金融政策の運営を困難にします。中央銀行は物価安定と経済成長のバランスを取る必要があるため、高インフレ下では金利引き上げによる金融引き締めが求められます。しかし、金利上昇は企業の資金調達コストを高め、設備投資の抑制を招きます。さらに、個人消費の減少にもつながるため、GDPの低迷を招きかねません。

このように、高インフレは消費、投資、貿易、金融政策など、経済の様々な側面に悪影響を及ぼします。その結果、GDPの伸び悩みや経済の停滞を招くのです。

したがって、政府は物価の安定化に向けた適切な政策対応が求められます。金融政策による需要管理のみならず、供給サイドの生産性向上や規制緩和など、総合的な経済政策の実施が重要となります。また、インフレ抑制と経済成長のバランスを取ることも課題です。高インフレからの脱却には長期的な取り組みが必要不可欠です。
Input tokens:  61
Output tokens:  716
Total tokens:  777
Stop reason: end_turn
Finished generating text with model anthropic.claude-3-haiku-20240307-v1:0.


In [19]:
output_message['content']

[{'text': '高インフレが国のGDPに与える影響について、1000文字でお書きします。\n\n高インフレは、国のGDPに深刻な影響を及ぼします。まず、高インフレは消費者の購買力を低下させます。物価が上昇すれば、同じ金額で以前より少ない商品しか購入できなくなります。これにより、個人消費が減少し、GDP成長率の鈍化につながります。\n\nまた、高インフレは企業の生産活動にも悪影響を及ぼします。原材料や人件費の高騰により、企業の利益率が低下します。利益が減少すれば、企業は設備投資の抑制や雇用調整を余儀なくされ、経済全体の生産性が低下します。\n\nさらに、高インフレは国際競争力の低下にもつながります。自国通貨の価値が下落すれば、輸出製品の価格が相対的に高くなり、海外市場での競争力が失われます。一方で、輸入品の価格が上昇するため、国内需要も減少します。これらの要因により、貿易収支の悪化を招き、GDPの減少につながります。\n\n加えて、高インフレは金融政策の運営を困難にします。中央銀行は物価安定と経済成長のバランスを取る必要があるため、高インフレ下では金利引き上げによる金融引き締めが求められます。しかし、金利上昇は企業の資金調達コストを高め、設備投資の抑制を招きます。さらに、個人消費の減少にもつながるため、GDPの低迷を招きかねません。\n\nこのように、高インフレは消費、投資、貿易、金融政策など、経済の様々な側面に悪影響を及ぼします。その結果、GDPの伸び悩みや経済の停滞を招くのです。\n\nしたがって、政府は物価の安定化に向けた適切な政策対応が求められます。金融政策による需要管理のみならず、供給サイドの生産性向上や規制緩和など、総合的な経済政策の実施が重要となります。また、インフレ抑制と経済成長のバランスを取ることも課題です。高インフレからの脱却には長期的な取り組みが必要不可欠です。'}]

In [18]:
from pprint import pprint
pprint(response)

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '2481',
                                      'content-type': 'application/json',
                                      'date': 'Fri, 31 May 2024 00:51:31 GMT',
                                      'x-amzn-requestid': '864b77d8-6212-4707-8600-e9d328baa31b'},
                      'HTTPStatusCode': 200,
                      'RequestId': '864b77d8-6212-4707-8600-e9d328baa31b',
                      'RetryAttempts': 0},
 'metrics': {'latencyMs': 13585},
 'output': {'message': {'content': [{'text': '高インフレが国のGDPに与える影響について、1000文字でお書きします。\n'
                                             '\n'
                                             '高インフレは、国のGDPに深刻な影響を及ぼします。まず、高インフレは消費者の購買力を低下させます。物価が上昇すれば、同じ金額で以前より少ない商品しか購入できなくなります。これにより、個人消費が減少し、GDP成長率の鈍化につながります。\n'
                                             '\n'
                                             'また、高インフレは企業の生産活動にも悪影響を

In [ ]:
new_user_input = {
    "role": "user",
    "content": [{"text": "具体例を教えてください。"}]
}

In [5]:
!pip list | grep boto3

boto3                                    1.34.116


In [2]:
!pip install -U boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 26.2 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.94
    Uninstalling botocore-1.34.94:
      Successfully uninstalled botocore-1.34.94
  Attempting uninstall: boto3
    Found existing installation: boto3 1.34.94
    Uninstalling boto3-1.34.94:
      Successfully uninstalled boto3-1.34.94
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.93 requires botocore==1.34.93, but you have botocore 1.34.116 which is incompatible.


In [ ]:
pip install "boto3>=1.34.116"

In [ ]:
https://aws.amazon.com/jp/about-aws/whats-new/2024/05/amazon-bedrock-new-converse-api/


In [14]:
import boto3
import json
brt = boto3.client(service_name='bedrock-runtime', region_name='us-west-2')

body = json.dumps({
    "prompt": "\n\nHuman: explain black holes to 8th graders\n\nAssistant:",
    "max_tokens_to_sample": 300,
    "temperature": 0.1,
    "top_p": 0.9,
})

modelId = 'anthropic.claude-v2'
accept = 'application/json'
contentType = 'application/json'

response = brt.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)

response_body = json.loads(response.get('body').read())

# text
print(response_body.get('completion'))

 Here is a simple explanation of black holes for 8th graders:

- A black hole is a region in space where gravity is so strong that nothing can escape from it, even light. This region is called the "event horizon."

- Black holes form when very massive stars collapse. When the star runs out of nuclear fuel, gravity makes it collapse. The matter gets compressed into a very small space, creating a black hole. 

- Black holes are called "black" because they absorb all the light that hits them. They don't reflect or emit any light that we can detect. So they appear completely black.

- The gravitational pull of black holes is so strong because matter has been compressed into a tiny space. All of that mass gets concentrated into a single point called the "singularity." 

- Not even light can escape a black hole once it crosses the event horizon. The gravity is so intense that the escape velocity is faster than the speed of light!

- Black holes warp space and time around them. They bend ligh

In [15]:
pprint(response)

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '1472',
                                      'content-type': 'application/json',
                                      'date': 'Fri, 31 May 2024 00:28:56 GMT',
                                      'x-amzn-bedrock-input-token-count': '17',
                                      'x-amzn-bedrock-invocation-latency': '15663',
                                      'x-amzn-bedrock-output-token-count': '300',
                                      'x-amzn-requestid': '150ae95e-f617-478e-bb82-477e03b3b51f'},
                      'HTTPStatusCode': 200,
                      'RequestId': '150ae95e-f617-478e-bb82-477e03b3b51f',
                      'RetryAttempts': 0},
 'body': <botocore.response.StreamingBody object at 0x7b504ee0c1f0>,
 'contentType': 'application/json'}


In [16]:
pprint(response_body)

{'completion': ' Here is a simple explanation of black holes for 8th graders:\n'
               '\n'
               '- A black hole is a region in space where gravity is so strong '
               'that nothing can escape from it, even light. This region is '
               'called the "event horizon."\n'
               '\n'
               '- Black holes form when very massive stars collapse. When the '
               'star runs out of nuclear fuel, gravity makes it collapse. The '
               'matter gets compressed into a very small space, creating a '
               'black hole. \n'
               '\n'
               '- Black holes are called "black" because they absorb all the '
               "light that hits them. They don't reflect or emit any light "
               'that we can detect. So they appear completely black.\n'
               '\n'
               '- The gravitational pull of black holes is so strong because '
               'matter has been compressed into a tiny sp